In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import os 

In [5]:
os.makedirs("data", exist_ok=True)

### Датасет 1: идеальные данные

In [6]:
X1 = np.linspace(0, 10, 100).reshape(-1, 1)
y1 = 3 * X1.flatten() + 5
df1 = pd.DataFrame({'x': X1.flatten(), 'y': y1})
df1.to_csv('data/clean.csv', index=False)

### Датасет 2: легкий шум

In [ ]:
y2 = 3 * X1.flatten() + 5 + np.random.normal(0, 1, size=X1.shape[0])
df2 = pd.DataFrame({'x': X1.flatten(), 'y': y2})
df2.to_csv('data/little_noise.csv', index=False)

### Датасет 3: линейные данные с шумом и выбросами

In [8]:
y3 = 3 * X1.flatten() + 5 + np.random.normal(0, 3, size=X1.shape[0])
df3 = pd.DataFrame({'x': X1.flatten(), 'y': y3})
df3.to_csv('data/noisy.csv', index=False)

In [9]:
model = LinearRegression()
model.fit(X1, y1)

import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [17]:
%%writefile test_model.py
import pandas as pd
import pickle
from sklearn.metrics import mean_squared_error, r2_score

with open("model.pkl", "rb") as f:
    model = pickle.load(f)


def evaluate(dataset_path):
    df = pd.read_csv(dataset_path)
    X = df[["x"]]
    y = df[["y"]]
    y_pred = model.predict(X)
    return mean_squared_error(y, y_pred), r2_score(y, y_pred)


def test_clean():
    mse, r2 = evaluate("data/clean.csv")
    assert mse < 1e-6, f"MSE слишком большая для clean.csv: {mse}"
    assert r2 > 0.9999, f"R2 слишком низний для clean.csv: {r2}"


def test_little_noise():
    mse, r2 = evaluate("data/little_noise.csv")
    assert (
        mse < 2.0
    ), f"MSE слишком большая для little_noise.csv с небольшим шумом: {mse}"
    assert r2 > 0.98, f"R2 слишком низкий для little_noise.csv с небольшим шумом: {r2}"


def test_noisy():
    mse, r2 = evaluate("data/noisy.csv")
    assert mse > 5.0, f"MSE неожиданно мала для noisy.csv: {mse}"
    assert r2 < 0.95, f"R2 слишком высок для noisy.csv: {r2}"


Overwriting test_model.py


In [18]:
!pytest test_model.py

============================= test session starts ==============================
platform darwin -- Python 3.13.3, pytest-8.3.5, pluggy-1.5.0
rootdir: /Users/aleksejsmolic/dev/urfu/mlops_practice/lab5
collected 3 items                                                              

test_model.py ...                                                        [100%]

=============================== warnings summary ===============================
test_model.py::test_clean
test_model.py::test_little_noise
test_model.py::test_noisy
  /Users/aleksejsmolic/dev/urfu/mlops_practice/lab5/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
    warnings.warn(

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
======================== 3 passed, 3 warnings in 1.23s =========================
